# OLD - CODE - DO NOT USE #

# Please see sample-with-32minsTrainedData.ipynb

# or see sample-with-6hoursTrainedData.ipynb

In [ ]:
from NSynthDataSet_RawAudio import NSynthDataSet_RawAudio
from transformers import GTransformer

import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.distributions as dist
from torch.utils.data import RandomSampler

import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import librosa

import random, sys, math, gzip, os
from tqdm import tqdm
from IPython.display import Audio 
import soundfile 

In [ ]:
### Constants 

sample_rate = 16000

batch_size = 16
lr = 0.001
lr_warmup = 5000
epochs = 100

sample_length = 512 # For context
embedding_size = 128 
num_heads = 8 # Number of chunks for 'parallel/ensemble' computation
depth = 12 # Number of transformer layers
num_tokens = 256 #Size of the dictionary

sample_index = 11
use_temperature = True
temperature = 0.5

In [ ]:
model = GTransformer(emb=embedding_size, heads=num_heads, depth=depth, seq_length=sample_length, num_tokens=num_tokens, attention_type=None)
model = model.cuda()

opt = torch.optim.Adam(lr=lr, params=model.parameters())
sch = torch.optim.lr_scheduler.LambdaLR(opt, lambda i: min(i / (lr_warmup / batch_size), 1.0))
loss = torch.nn.NLLLoss(reduction='mean')

In [ ]:
print(model)

In [ ]:
base_data_dir = '/home/purnima/appdir/Github/DATA/NSynth/'
labels_dir = '/home/purnima/appdir/Github/DATA/NSynth'

validate_data_dir = os.path.join(base_data_dir,'nsynth-valid','audio')
labels_validate_dir = os.path.join(labels_dir,'nsynth-valid', 'examples.json')

validate_ds = NSynthDataSet_RawAudio(meta_data_file=labels_validate_dir, audio_dir=validate_data_dir, sr=sample_rate)
validate_loader = torch.utils.data.DataLoader(validate_ds, batch_size=batch_size, shuffle=False)

## Sampling
Sampling is done as follows -  
1. Select an input from the dataloader (update sample_index to any value from 0 to 15). Shape will be `1 X sample_length` or `1 X 512`
2. Generate the output from the model. Shape will be `1 X 512 X 256`
3. Retrieve the last sample from the 512 sample_length dimension `sample_data = sample_data[0, -1, :]` (argmax or temperature sampling (-- I need to understand this more--)  
4. Append this sample to the input (from step 1). Both input and this new sample are between 0 - 255. Use this as the new input and repeat from Step 1 for as long as you want (here we are iterating through the steps 256 times to generate 256 new samples).
5. Convert this 512 + 256 new samples to waveform using mu_expand from librosa.

In [ ]:
def sample():
    iterator = iter(validate_loader)
    orig_test_data, _ = iterator.next()

    test_data = orig_test_data[sample_index].view(1,-1)
    orig_waveform = mulawDecode(test_data.view(-1).cpu())
    test_data_ = test_data.detach().clone().cuda()
    test_data = test_data.detach().clone().cuda()
    
    with torch.no_grad():
        for ind in range(int(sample_length) * 10):
            sample_data = model(test_data) 
            if use_temperature:
                temperature = 0.2
                p = F.softmax(sample_data / temperature, dim=0)
                cd = dist.Categorical(p)
                sample_data = cd.sample()
                sample_data = sample_data[:,-1:]
            else:
                sample_data = sample_data[0, -1, :].argmax() 
            
            sample_data = sample_data.view(1,-1)
            test_data_ = torch.cat((test_data_, sample_data), dim=1)
            test_data = test_data_[:,test_data_.shape[1]-512:].view(1,-1)
    
    test_data_ = test_data_.view(-1).cpu()
    waveform = mulawDecode(test_data_)
    
    return waveform, orig_waveform

In [ ]:
def load_model(model, opt, model_location):
    checkpoint = torch.load(model_location)
    model.load_state_dict(checkpoint['model_state_dict'])
    opt.load_state_dict(checkpoint['optimizer_state_dict'])
    loss = checkpoint['loss']
    epoch = checkpoint['epoch']
    return model, opt, loss, epoch

def mulawDecode(output):
    output = output.numpy() - 127
    waveform = librosa.mu_expand(output, quantize=True)
    return waveform

In [ ]:
model, opt, loss, epoch = load_model(model, opt, 'checkpoint-gradclip-emblayer/attention-10.pt')
print('Model Loaded')


In [ ]:
use_temperature = False
new_sample, orig_seed = sample()

In [ ]:
print('Original seed')
plt.plot(orig_seed)
soundfile.write('samples/orig_seed.wav', orig_seed, samplerate=sample_rate)
Audio(orig_seed, rate=sample_rate)


In [ ]:
print('New sample appended to original seed') #First 512 samples are original seed
plt.plot(new_sample)
soundfile.write('samples/new_sample.wav', new_sample, samplerate=sample_rate)
print(len(new_sample))
Audio(new_sample, rate=sample_rate)


In [ ]:
2.5*16000

In [ ]:
audio_data, _ = librosa.load('/home/purnima/appdir/Github/DATA/NSynth/nsynth-valid/audio/guitar_acoustic_010-084-100.wav', sr=sample_rate)
print(_, len(audio_data))
plt.plot(audio_data)
audio_data = audio_data[int(3.125*16000):int(3.125*16000)+512] 
plt.figure()
plt.plot(audio_data)


In [ ]:
audio_data = librosa.mu_compress(audio_data, quantize=True) + 127
audio_data = audio_data.astype(np.long)
audio_data = torch.from_numpy(audio_data)

test_data = audio_data.view(1,-1)
orig_waveform = mulawDecode(test_data.view(-1).cpu())
test_data_ = test_data.detach().clone().cuda()
test_data = test_data.detach().clone().cuda()

with torch.no_grad():
    for ind in range(int(sample_length) * 10):
        sample_data = model(test_data) 
        if use_temperature:
            temperature = 0.2
            p = F.softmax(sample_data / temperature, dim=0)
            cd = dist.Categorical(p)
            sample_data = cd.sample()
            sample_data = sample_data[:,-1:]
        else:
            sample_data = sample_data[0, -1, :].argmax() 

        sample_data = sample_data.view(1,-1)
        test_data_ = torch.cat((test_data_, sample_data), dim=1)
        test_data = test_data_[:,test_data_.shape[1]-512:].view(1,-1)

test_data_ = test_data_.view(-1).cpu()
waveform = mulawDecode(test_data_)


In [ ]:
print('Original seed')
plt.plot(orig_waveform)
soundfile.write('samples/orig_seed.wav', orig_seed, samplerate=sample_rate)
Audio(orig_seed, rate=sample_rate)


In [ ]:
print('New Sample')
plt.plot(waveform)
soundfile.write('samples/new_sample.wav', waveform, samplerate=sample_rate)
Audio(waveform, rate=sample_rate)

In [ ]:
from librosa.display import specshow
D = librosa.stft(orig_seed)  # STFT of y
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
fig, ax = plt.subplots()
img = librosa.display.specshow(S_db, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='Orig seed')
fig.colorbar(img, ax=ax, format="%+2.f dB")

In [ ]:
D = librosa.stft(waveform)  # STFT of y
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
fig, ax = plt.subplots()
img = librosa.display.specshow(S_db, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='New Sample')
fig.colorbar(img, ax=ax, format="%+2.f dB")

In [ ]:
audio_data, _ = librosa.load('samples/new_sample-cool.wav')
audio_data = audio_data * 50
plt.plot(audio_data)
Audio(audio_data, rate=sample_rate)

In [ ]:
D = librosa.stft(audio_data)  # STFT of y
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
fig, ax = plt.subplots()
img = librosa.display.specshow(S_db, x_axis='time', y_axis='linear', ax=ax)
ax.set(title='New Sample')
fig.colorbar(img, ax=ax, format="%+2.f dB")